# Objetos na Memória (parte 2)

Este notebook contém exemplos do livro [_Fluent Python, Second Edition_](https://www.fluentpython.com/) .

## Identidade, igualdade, e aliases

> **NOTA:** A palavra inglesa *alias* pode ser traduzida como *apelido* ou *alcunha*.

Pelé era o apelido de Edson Arantes do Nascimento.
Pelé não é apenas igual a Edson; os dois nomes se referem à mesma pessoa.

Veja esta ideia em Python:

In [ ]:
edson = dict(nome='Edson Arantes do Nascimento', ano=1940)
edson

In [ ]:
pelé = edson
pelé is edson

In [ ]:
id(pelé), id(edson)

In [ ]:
pelé['gols'] = 1283
edson

Agora imagine um impostor, tentando se passar por Pelé.
O impostor alega que tem os mesmos dados:

In [ ]:
impostor = {'nome': 'Edson Arantes do Nascimento', 'ano': 1940, 'gols': 1283}

In [ ]:
impostor is pelé

In [ ]:
impostor == pelé

In [ ]:
impostor is not pelé

## Como escolher entre `==` ou `is`

Se `a is b` é verdadeiro, então `a == b` também é verdeiro, pois `a` e `b` são referências para o mesmo objeto.

Mas o contrário nem sempre é verdade:
é possível que `a == b` seja verdade, mesmo quando `a is b` é falso.
Isso acontece porque dois objetos distintos na memória podem ter igual valor.

Por exemplo, duas listas distintas podem ter o mesmo tamanho e os itens de igual valor.

In [ ]:
a = [1, 2, 3]
b = [1.0, 2.0, 3.0]
a == b

In [ ]:
a is b

Para comparar *valores*, e não *identidades*, sempre use `==`.
É muito mais comum a gente comparar o *valor* de dois objetos do sua *identidade* deles.

O caso mais comum de uso de `is` é comparar com objetos únicos (singletons),
geralmente usados como sinalizadores ou sentinelas.
Por exemplo: `x is None`.
Se `x == None` nesse caso é garantido que `x is None` também é verdade,
porque só existe um objeto `None`.

In [ ]:
x = None
y = None
x is y  # só existe um objeto None

In [ ]:
x is None

In [ ]:
x == None  # também funciona, mas é mais lento

Quando um objeto é único é mais eficiente verificar a identidade do que a igualdade,
porque o operador `==` pode ser sobrecarregado implementando o método `__eq__`,
portanto toda vez que aparece `==`, o interpretador precisa verificar a presença do método `__eq__`.

Mas o operador `is` não pode ser sobrecarregado.
Ele é implementado em C simplesmente comparando o *id* dos objetos.
Por isso ele é mais rápido.

### Exemplo de `is` com `None`

In [ ]:
from unicodedata import name

for código in range(0x30):
    car = chr(código)
    nome = name(car, None)
    if nome is None:
        continue
    print(f'U+{código:04x}\t{car}\t{nome}')

### Exemplo de `is` com sentinela

Sentinelas são valores especiais usados em filas ou sockets
para sinalizar o fim de uma sequência de valores.

A forma mais simples de criar uma sentinela única é instanciar `object()`.
Lembre-se que cada objeto tem sua própria identidade.

In [ ]:
FIM_DA_SÉRIE = object()
BANANA = object()

FIM_DA_SÉRIE == BANANA, FIM_DA_SÉRIE is BANANA

Agora vamos usar o objeto `FIM_DA_SÉRIE` como uma sentinela.

Ao preencher uma fila, a função `carregar` coloca `FIM_DA_SÉRIE`
após o último item carregado.

Assim, o código-cliente sabe quando parar de retirar uma série de itens da fila.

In [ ]:
from queue import SimpleQueue
from collections.abc import Iterable


def carregar(fila: SimpleQueue, itens: Iterable) -> None:
    for item in itens:
        fila.put(item)
    fila.put(FIM_DA_SÉRIE)
    
   
fila = SimpleQueue()
carregar(fila, 'ABCDZ')

# código-cliente: consome itens da fila até encontrar FIM_DA_SÉRIE 
    
while (item := fila.get()) is not FIM_DA_SÉRIE:
    print(item)  

> **NOTA:** Filas são muito usadas para sincronizar e trocar dados entre thread: uma thread coloca itens na fila, e outra thread consome itens da fila. A sentinela permite que a thread produtora informe a thread consumidora quando uma série de itens está completa.

----

### 🤔 Operador Morsa `:=`

O laço **`while`** acima usa o "operador morsa" `:=` introduzido no Python 3.8.

O exemplo acima antes do Python 3.8 seria escrito assim:

In [ ]:
fila = preparar()

while True:
    item = fila.get()
    if item is FIM_DA_SÉRIE:
        break
    print(item)

Chama-se "operador morsa" por causa do emoticon `:=` que representa uma morsa:

<img src="https://upload.wikimedia.org/wikipedia/commons/c/ce/Noaa-walrus22.jpg" width="300">

----